After failing with Deep Neural Network using Sarsa I needed to make sure I could do this without func approx and lo and behold, this took me like literally just 1 day

In [8]:
import numpy as np
from collections import deque
import time

In [2]:
class board:
    
    def __init__(self):    
        #Game specific Variables
        self.turn = 1
        self.moves_left = 9
        self.state = np.array([[0 for _ in range(9)]])
        self.mask = np.array([[1 for _ in range(9)]])
        
    def update(self, action):
        
        if self.state[0][action] != 0:
            raise ValueError('invalid move!', self.state, self.turn, self.moves_left, action)
            return 'invalid move'
        
        self.state[0][action] = self.turn
        self.mask[0][action] = 0
        self.turn = (self.turn == 1)*(-1)+(self.turn == -1)*(1)
        self.moves_left -= 1
        
        if self.moves_left < 5:
            win_cond = -3*self.turn
                        
            # row check
            temp = action//3 * 3
            if self.state[0][temp] + self.state[0][temp+1] + self.state[0][temp+2] == win_cond:
                return f"{-1*self.turn} won"
            
            # col check
            if self.state[0][action] + self.state[0][(action+3)%9] + self.state[0][(action+6)%9] == win_cond:
                return f"{-1*self.turn} won"
            
            #diag check
            if action%2 == 0:
                if self.state[0][0] + self.state[0][4] + self.state[0][8] == win_cond \
                or self.state[0][2] + self.state[0][4] + self.state[0][6] == win_cond:
                    return f"{-1*self.turn} won"
                            
            if self.moves_left == 0:
                return 'tie'
        
        return 'valid move'

    def reset(self):
        self.turn = 1
        self.moves_left = 9
        self.state = np.array([[0 for _ in range(9)]])
        self.mask = np.array([[1 for _ in range(9)]])

class game:
    
    def __init__(self):
        #RL Variables
        self.terminal = False
        self.reward = 0
        
        #Tic Tac Toe Board
        self.gameboard = board()
        
    def update(self, action):                    
        result = self.gameboard.update(action)
        
        if result == 'invalid move':
            self.reward = -1
            self.terminal = True
            
        elif result == 'valid move':
            self.reward = 0
            self.terminal = False
            
        elif result == '1 won':
            self.reward = 1 * -1 * self.gameboard.turn
            self.terminal = True
            
        elif result == '-1 won':
            self.reward = -1 * -1 * self.gameboard.turn
            self.terminal = True
        
        elif result == 'tie':
            self.reward = 0
            self.terminal = True
                
    def reset(self):
        self.gameboard.reset()
        
        self.terminal = False
        self.reward = 0
    
    def return_state_features(self):
        return self.gameboard.state.reshape(1,9)*self.gameboard.turn
    
    def get_mask(self):
        return self.gameboard.mask
    
    def get_reward(self):
        return self.reward
    
class illustrator:
    
    def __init__(self, host_game):
        self.host = host_game
    
    def draw(self, state=None):
        if state is not None:
            st = state
        else:
            st = self.host.gameboard.state*self.host.gameboard.turn
        
        def convert(n):
            return "X"*(n==1) + " "*(n==0) + "O"*(n==-1)
        temp = [[convert(int(st[0][3*j+i])) for i in range(3)] for j in range(3)]
        
        print(f"{temp[0][0]}|{temp[0][1]}|{temp[0][2]}")
        print("-----")
        print(f"{temp[1][0]}|{temp[1][1]}|{temp[1][2]}")
        print("-----")
        print(f"{temp[2][0]}|{temp[2][1]}|{temp[2][2]}")

In [3]:
class player:
    def __init__(self):
        self.epsilon = 0
        
    def q_values(self, s):
        return "IS A PLAYER"
    
    def choose_action(self, s, m):
        return int(input("Where would you like to move? "))

In [4]:
class random_bot:
    def __init__(self):
        self.epsilon = 0
        
    def q_values(self, s):
        return "IS A RANDOM BOT"
    
    def choose_action(self, s, m):
        return (np.random.uniform(low = 0.5, high = 1, size=[1,9]) * m).argmax()

In [5]:
class sarsa_simple:
    def __init__(self, num_states, num_actions, alpha, epsilon, gamma, decay=0.99999):
        self.q_table = np.zeros((num_states, num_actions))
        self.alpha = alpha
        self.epsilon = epsilon
        self.min_epsilon = 0.1
        self.decay = decay
        self.gamma = gamma
        
    def pre_process(state_features):
        st = 0
        for i in range(9):
            st+= 3**i * (state_features[0][i]+1)
        return st
    
    def q_values(self, state_features):
        return self.q_table[sarsa_simple.pre_process(state_features)]
        
    def choose_action(self, state_features, mask):
        
        if self.epsilon > self.min_epsilon:
            self.epsilon*=self.decay
        
        if np.random.rand() < self.epsilon:
            p = np.random.uniform(size=(1,9)) * mask
            return int(p.argmax())
        
        q_values = self.q_values(state_features)
        q_values = (q_values + 1) * mask
        
        return int(q_values.argmax())
        
    def update_normal(self, s, a, r, s2, a2):
        s = sarsa_simple.pre_process(s)
        s2 = sarsa_simple.pre_process(s2)
        self.q_table[s][a] += self.alpha*(r+self.gamma*self.q_table[s2][a2]-self.q_table[s][a])
    
    def update_final(self, s, a, r):
        s = sarsa_simple.pre_process(s)
        self.q_table[s][a] += self.alpha*(r-self.q_table[s][a])
        

In [6]:
import json 

def save_simple(agent, filename = 'simple_agent'):
    
    data = {"q_table": agent.q_table.tolist(),
            "gamma": agent.gamma,
            "alpha": agent.alpha,
            "epsilon": agent.epsilon,
            "decay": agent.decay}
    
    with open(filename, "w") as f:
        json.dump(data, f)

def load_simple(filename = 'simple_agent'):
    
    with open(filename, "r") as f:
        data = json.load(f)
    
    agent = sarsa_simple(1,1,0,0,0)
    
    agent.q_table = np.array(data["q_table"])
    agent.alpha = data['alpha']
    agent.gamma = data['gamma']
    agent.epsilon = data['epsilon']
    agent.decay = data['decay']
    return agent

In [12]:
b = game()
i = illustrator(b)
agent = sarsa_simple(19683, 9, 0.05, 1, 0.9, 0.9991)
start_time = time.perf_counter()

training_steps = 100000
for step in range(training_steps):
    if step%100==0:
        print(f"{step} steps are done! Agent epsilon: {agent.epsilon}   ", end='\r')
    
    s1, m = b.return_state_features(), b.get_mask()
    a1 = agent.choose_action(s1, m) 
    b.update(a1)
    r1 = b.get_reward()

    s2, m = b.return_state_features(), b.get_mask()
    a2 = agent.choose_action(s2, m) 
    b.update(a2)
    r2 = b.get_reward()

    remember = deque([(s1, a1, r1), (s2, a2, r2)])

    while not b.terminal:

        s, m = b.return_state_features(), b.get_mask()
        a = agent.choose_action(s, m)
        b.update(a)
        r = b.get_reward()

        remember.append((s,a,r))
        sb, ab, rb = remember.popleft()
        agent.update_normal(sb, ab, rb, s, a)

    sb2, ab2, _ = remember.popleft() #Idiot action
    sb1, ab1, r = remember.popleft() #Winner boy
    agent.update_final(sb2, ab2, -1*r)
    agent.update_final(sb1, ab1, r)
    
    b.reset()
    
    if step == training_steps - 1: 
        print(f"{step} steps are done! Agent epsilon: {agent.epsilon}   ")
    
save_simple(agent, 'agent_{}'.format(training_steps))
print(f"Training finished. Took {time.perf_counter() - start_time} seconds")

99999 steps are done! Agent epsilon: 0.09993486931725679   
Training finished. Took -47.2914116 seconds


In [13]:
b = game()
i = illustrator(b)
player1 = load_simple('agent_10000')#
player1 = load_simple('agent_{}'.format(training_steps))
player2 = player()#player2 = load_simple('good_agent')

init_e1, init_e2 = player1.epsilon, player2.epsilon
player1.epsilon, player2.epsilon = 0, 0

while not b.terminal:
    s, m = b.return_state_features(), b.get_mask()
    print(f"player1 thinks values are {player1.q_values(s)}")
    i.draw()
    a = player1.choose_action(s, m)
    b.update(a)
    r = b.get_reward()
    print(f"action was {a} and the reward was {r}\n")
    

    if not b.terminal:
        s, m = b.return_state_features(), b.get_mask()
        print(f"player2 thinks values are {player2.q_values(s)}")
        i.draw()
        a = player2.choose_action(s, m)
        b.update(a)
        r = b.get_reward()
        print(f"action was {a} and the reward was {r}\n")
            
player1.epsilon, player2.epsilon = init_e1, init_e2
i.draw()

player1 thinks values are [ 0.00755753  0.04452375  0.01543171  0.00230985  0.08720691 -0.00197139
  0.00100311  0.01238209  0.00348525]
 | | 
-----
 | | 
-----
 | | 
action was 4 and the reward was 0

player2 thinks values are [-0.22853082 -0.68060082 -0.17105761 -0.63959181  0.         -0.69608456
 -0.23204509 -0.69973772 -0.27887964]
 | | 
-----
 |O| 
-----
 | | 
action was 2 and the reward was 0

player1 thinks values are [0.05822488 0.05409265 0.         0.03412084 0.         0.04989984
 0.02719885 0.03442169 0.03167058]
 | |O
-----
 |X| 
-----
 | | 
action was 0 and the reward was 0

player2 thinks values are [ 0.         -0.99412706  0.         -0.94052659  0.         -0.84120004
 -0.9713807  -0.95150947  0.03054105]
O| |X
-----
 |O| 
-----
 | | 
action was 8 and the reward was 0

player1 thinks values are [ 0.         -0.73555571  0.         -0.67646646  0.          0.10177298
 -0.74255199 -0.6739063   0.        ]
X| |O
-----
 |X| 
-----
 | |O
action was 5 and the reward was 0


In [16]:
b = game()
i = illustrator(b)

player1 = random_bot()#load_simple('good_agent')
player2 = load_simple('agent_300000')

init_e1, init_e2 = player1.epsilon, player2.epsilon
expl_rate = 0
player1.epsilon, player2.epsilon = expl_rate, expl_rate 

num_matches = 20000
p1_wins = 0
p2_wins = 0
for i in range(num_matches):
    
    while not b.terminal:
        s, m = b.return_state_features(), b.get_mask()
        a = player1.choose_action(s, m)
        b.update(a)
        r = b.get_reward()
        if r == 1:
            p1_wins +=1


        if not b.terminal:
            s, m = b.return_state_features(), b.get_mask()
            a = player2.choose_action(s, m)
            b.update(a)
            r = b.get_reward()
            if r == 1:
                p2_wins+=1
                    
    b.reset()
    
    if i%100==0:
        msg1 = f'player 1 {str(player1.__class__)[17:-2]} won {p1_wins} times'
        msg2 = f' and player 2 {str(player2.__class__)[17:-2]} won {p2_wins} times.' 
        msg3 = f' They drew {i-p1_wins-p2_wins} times.'
        msg4 = f' They played {i} games.    '
        print(msg1 + msg2 + msg3 + msg4 + '        ', end = '\r')
    #print(f"hi {i}", end='\r') 
player1.epsilon, player2.epsilon = init_e1, init_e2